In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/My Drive/semeval_2022/dontpatronizeme_v1.3

/content/drive/My Drive/semeval_2022/dontpatronizeme_v1.3


In [3]:
!pip install beautifulsoup4 


In [4]:
!pip install transformers

     |████████████████████████████████| 3.5 MB 14.2 MB/s 
     |████████████████████████████████| 67 kB 6.7 MB/s 
     |████████████████████████████████| 6.8 MB 8.3 MB/s 
     |████████████████████████████████| 895 kB 68.0 MB/s 
     |████████████████████████████████| 596 kB 70.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
import os
import re
import numpy as np
import string
import json
#import emoji
import pandas as pd
from sklearn import metrics
from bs4 import BeautifulSoup
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, AutoTokenizer, BertModel, BertConfig, AutoModel, AdamW,RobertaModel, RobertaConfig,DebertaTokenizer, DebertaModel
pd.set_option("display.max_columns", None)
import random

In [6]:
random.seed(55)
np.random.seed(55)
torch.manual_seed(55)
torch.cuda.manual_seed_all(55)

In [ ]:
test_df=pd.read_csv('task4_test.csv')

In [7]:
train_df=pd.read_csv('task2_train.csv')
val_df=pd.read_csv('task2_test.csv')
#train_df=pd.concat([train_df,val_df])

In [ ]:
train_df.reset_index(inplace=True)

In [ ]:
len(train_df)

993

In [ ]:
from ast import literal_eval
train_df=pd.read_csv('task2_redefined_train.csv')
val_df=pd.read_csv('task2_redefined_test.csv')
train_df.label = train_df.label.apply(literal_eval)
val_df.label=val_df.label.apply(literal_eval)

In [8]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", 
                       "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", 
                       "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", 
                       "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am",
                       "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", 
                       "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have",
                       "it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not",
                       "mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", 
                       "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                       "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", 
                       "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have",
                       "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is",
                       "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would",
                       "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have",
                       "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have",
                       "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", 
                       "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did",
                       "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", 
                       "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", 
                       "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                       "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have",
                       "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have", 'u.s':'america', 'e.g':'for example'}

punct = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-",
                 "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 
                 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', '!':' '}

mispell_dict = {'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater',
                'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ',
                'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can',
                'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 
                'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 
                'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 
                'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization',
                'demonetisation': 'demonetization'}

In [9]:
def clean_text(text):
    '''Clean emoji, Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    #text = emoji.demojize(text)
    text = re.sub(r'\:(.*?)\:','',text)
    text = str(text).lower()    #Making Text Lowercase
    text = re.sub('\[.*?\]', '', text)
    #The next 2 lines remove html text
    text = BeautifulSoup(text, 'lxml').get_text()
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",", "'")
    text = re.sub(r"[^a-zA-Z?.!,¿']+", " ", text)
    return text

def clean_contractions(text, mapping):
    '''Clean contraction using contraction mapping'''    
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    for word in mapping.keys():
        if ""+word+"" in text:
            text = text.replace(""+word+"", ""+mapping[word]+"")
    #Remove Punctuations
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    text = re.sub(r"([?.!,¿])", r" \1 ", text)
    text = re.sub(r'[" "]+', " ", text)
    return text
def clean_special_chars(text, punct, mapping):
    '''Cleans special characters present(if any)'''   
    for p in mapping:
        text = text.replace(p, mapping[p])
    
    for p in punct:
        text = text.replace(p, f' {p} ')
    
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}  
    for s in specials:
        text = text.replace(s, specials[s])
    
    return text

def correct_spelling(x, dic):
    '''Corrects common spelling errors'''   
    for word in dic.keys():
        x = x.replace(word, dic[word])
    return x

def remove_space(text):
    '''Removes awkward spaces'''   
    #Removes awkward spaces 
    text = text.strip()
    text = text.split()
    return " ".join(text)

def text_preprocessing_pipeline(text):
    '''Cleaning and parsing the text.'''
    text = clean_text(text)
    text = clean_contractions(text, contraction_mapping)
    text = clean_special_chars(text, punct, punct_mapping)
    text = correct_spelling(text, mispell_dict)
    text = remove_space(text)
    return text

In [10]:
train_df['text'] = train_df['text'].astype(str)
val_df['text'] = val_df['text'].astype(str)
train_df['text'] = train_df['text'].apply(text_preprocessing_pipeline)
val_df['text'] = val_df['text'].apply(text_preprocessing_pipeline)

In [ ]:
test_df['text'] = test_df['text'].astype(str)
test_df['text'] = test_df['text'].apply(text_preprocessing_pipeline)

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [12]:
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 32

In [13]:
MAX_LEN = 128
EPOCHS = 10
LEARNING_RATE = 2e-5
#tokenizer = AutoTokenizer.from_pretrained("nghuyong/ernie-2.0-en")
#tokenizer=DebertaTokenizer.from_pretrained('microsoft/deberta-base')
tokenizer = AutoTokenizer.from_pretrained('bert-large-uncased')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [14]:
target_cols=['upr','ss','ps','av','m','c','pm']

In [15]:
class BERTDataset(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.df = df
        self.max_len = max_len
        self.text = df.text
        self.tokenizer = tokenizer
        self.targets = df[target_cols].values
        #self.targets = df.label
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        text = self.text[index]
        inputs = self.tokenizer.encode_plus(
            text,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]
        
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [16]:
class BERTDatasetest(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.df = df
        self.max_len = max_len
        self.text = df.text
        self.tokenizer = tokenizer
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        text = self.text[index]
        inputs = self.tokenizer.encode_plus(
            text,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]
        
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long)
        }

In [17]:
train_dataset = BERTDataset(train_df, tokenizer, MAX_LEN)
valid_dataset = BERTDataset(val_df, tokenizer, MAX_LEN)
train_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, 
                          num_workers=2, shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=VALID_BATCH_SIZE, num_workers=2, shuffle=False, pin_memory=True)

In [ ]:
test_dataset=BERTDatasetest(test_df,tokenizer,MAX_LEN)
test_loader = DataLoader(test_dataset, batch_size=VALID_BATCH_SIZE, 
                          num_workers=2, shuffle=False, pin_memory=True)

In [ ]:
config = RobertaConfig.from_json_file('./checkpoint-68000/config.json')

In [18]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        #self.roberta = AutoModel.from_pretrained("nghuyong/ernie-2.0-en")
        self.roberta = AutoModel.from_pretrained("bert-large-uncased")
        #self.roberta = RobertaModel.from_pretrained('./checkpoint-68000/',local_files_only=True,config=config)
        #self.l2 = torch.nn.Dropout(0.2)
        self.fc = torch.nn.Linear(1024,7)
    
    def forward(self, ids, mask, token_type_ids):
         _, features = self.roberta(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)
         output = self.fc(features)
         return output

model = BERTClass()
model.to(device)

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERTClass(
  (roberta): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise

In [ ]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 
##########DEBARTA###
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        #self.roberta = AutoModel.from_pretrained("nghuyong/ernie-2.0-en")
        #self.roberta = RobertaModel.from_pretrained('./checkpoint-68000/',local_files_only=True,config=config)
        self.roberta=DebertaModel.from_pretrained('microsoft/deberta-base')
        #self.l2 = torch.nn.Dropout(0.2)
        self.fc = torch.nn.Linear(768,7)
    
    def forward(self, ids, mask, token_type_ids):
        # _, features = self.roberta(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)
        features = self.roberta(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False) 
        features=self.pool_hidden_state(features)
        output = self.fc(features)
        return output
    def pool_hidden_state(self, last_hidden_state):
      """
      Pool the output vectors into a single mean vector 
      """
      last_hidden_state = last_hidden_state[0]
      mean_last_hidden_state = torch.mean(last_hidden_state, 1)
      return mean_last_hidden_state

model = BERTClass()
model.to(device)

Some weights of the model checkpoint at microsoft/deberta-base were not used when initializing DebertaModel: ['lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.weight']
- This IS expected if you are initializing DebertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERTClass(
  (roberta): DebertaModel(
    (embeddings): DebertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=0)
      (LayerNorm): DebertaLayerNorm()
      (dropout): StableDropout()
    )
    (encoder): DebertaEncoder(
      (layer): ModuleList(
        (0): DebertaLayer(
          (attention): DebertaAttention(
            (self): DisentangledSelfAttention(
              (in_proj): Linear(in_features=768, out_features=2304, bias=False)
              (pos_dropout): StableDropout()
              (pos_proj): Linear(in_features=768, out_features=768, bias=False)
              (pos_q_proj): Linear(in_features=768, out_features=768, bias=True)
              (dropout): StableDropout()
            )
            (output): DebertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): DebertaLayerNorm()
              (dropout): StableDropout()
            )
          )
          (intermediate): DebertaIntermedi

In [ ]:
checkpoint=torch.load('./saved_models/roberta_wo_eda_softmargin_with_weights.pt')
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [ ]:
from sklearn.utils import class_weight
NUM_CLASSES=7
labels=train_df[target_cols].values
cw_arr = np.empty([NUM_CLASSES, 2])
for i in range(NUM_CLASSES):
  cw_arr[i] = class_weight.compute_class_weight(class_weight='balanced', classes = [0,1], y=labels[:, i])
cw_arr=torch.from_numpy(cw_arr).to(device)
#print(cw_arr.size()[1])
cw_arr=cw_arr[:,1]

In [ ]:
cw_arr=torch.FloatTensor([3.26,0.39,4.18,3.48,1.15,4.18,21.68]).to(device)

In [ ]:
def corr_basic_calc(weights, y_pred):
    #num_classes = len(weights[:,0])
    m_c = []
    sum_m_c = []
    for i in range(7):
        m_c.append(y_pred[:,i])
        sum_m_c.append(torch.sum(m_c[i]))
    return torch.stack(m_c), torch.stack(sum_m_c)


In [ ]:
torch.mean(((weights[:,0]*(1-targets))+(weights[:,1]*targets))*torch.nn.BCEWithLogitsLoss()(outputs, targets),axis=1)

In [ ]:
    def get_label_frequencies(data):
        label_freqs = Counter()
        for _, row in data.iterrows():
            label_freqs.update(row["label"])
        return label_freqs

In [19]:
def focal_binary_cross_entropy(logits, targets, gamma=2):
    num_label=7
    l = logits.reshape(-1)
    t = targets.reshape(-1)
    p = torch.sigmoid(l)
    p = torch.where(t >= 0.5, p, 1-p)
    logp = - torch.log(torch.clamp(p, 1e-4, 1-1e-4))
    loss = logp*((1-p)**gamma)
    loss = num_label*loss.mean()
    return loss

In [ ]:
def loss_fn(outputs,targets,weights):
  return torch.nn.MultiLabelSoftMarginLoss(weight=weights)(outputs, targets)

In [ ]:
torch.nn.BCEWithLogitsLoss(pos_weight=weights)

In [ ]:
def corr_loss(weights, y_pred,uncorrelated_c_pairs):
    num_classes = len(weights[:,0])
   
    loss=0
    for uncorr_c_pair in uncorrelated_c_pairs:
          c1, c2, v = uncorr_c_pair
          loss += torch.sum(y_pred[:,c1]*y_pred[:,c2])*((y_pred[:,c1].sum()+y_pred[:,c2].sum()))/(y_pred[:,c1].sum()*y_pred[:,c2].sum())
 
    return loss

In [ ]:
def loss_fn(outputs, targets):
  loss=torch.nn.BCEWithLogitsLoss()(outputs, targets)
  return loss


In [20]:
optimizer = AdamW(params = model.parameters(), lr=LEARNING_RATE, weight_decay=1e-6)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [21]:

def train(epoch):
    model.train()
    for _, data in enumerate(train_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        loss = focal_binary_cross_entropy(outputs, targets)
        if _%500 == 0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

In [21]:
EPOCHS=10

In [22]:

for epoch in range(EPOCHS):
    train(epoch)

Epoch: 0, Loss:  1.8339132070541382
Epoch: 1, Loss:  0.7090386152267456
Epoch: 2, Loss:  0.6474050283432007
Epoch: 3, Loss:  0.5134425759315491
Epoch: 4, Loss:  0.38359636068344116
Epoch: 5, Loss:  0.30966606736183167
Epoch: 6, Loss:  0.27611109614372253
Epoch: 7, Loss:  0.16714252531528473
Epoch: 8, Loss:  0.08357606828212738
Epoch: 9, Loss:  0.08021131902933121


In [ ]:
import pickle

In [ ]:
with open('./saved_models/corr_fuzzy_0.1.pickle', 'rb') as f:
    uncorrelated_c_pairs = pickle.load(f)

In [ ]:
def validation():
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(valid_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            #print(outputs)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(outputs.cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [ ]:
outputs, targets = validation()

In [ ]:
outputs = np.clip(np.sign(np.array(outputs)), a_min=0,a_max=None)

In [ ]:
accuracy = metrics.accuracy_score(targets, outputs)
f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
print(f"Accuracy Score = {accuracy}")
print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

Accuracy Score = 0.25125628140703515
F1 Score (Micro) = 0.7001287001287001
F1 Score (Macro) = 0.5494787434460434


In [23]:
def validation():
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(valid_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [25]:
outputs, targets = validation()
outputs = np.array(outputs) >= 0.5
accuracy = metrics.accuracy_score(targets, outputs)
f1_score_micro = metrics.f1_score(targets, outputs, average='micro')
f1_score_macro = metrics.f1_score(targets, outputs, average='macro')
print(f"Accuracy Score = {accuracy}")
print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

Accuracy Score = 0.33668341708542715
F1 Score (Micro) = 0.725219573400251
F1 Score (Macro) = 0.5848379871905712


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(targets, outputs))

              precision    recall  f1-score   support

           0       0.53      0.57      0.55        44
           1       0.78      0.96      0.86       141
           2       0.71      0.48      0.58        31
           3       0.72      0.51      0.60        45
           4       0.90      0.69      0.78       101
           5       0.65      0.28      0.39        39
           6       0.00      0.00      0.00         6

   micro avg       0.75      0.69      0.72       407
   macro avg       0.61      0.50      0.54       407
weighted avg       0.75      0.69      0.70       407
 samples avg       0.79      0.72      0.71       407



In [ ]:
state={'epoch': epoch, 'state_dict':model.state_dict(),'optimizer': optimizer.state_dict()}

In [ ]:
torch.save(state, './saved_models/robertalarge_focalfinal_10_128.pt')

In [ ]:
###########prediction#########

In [ ]:
def test():
    model.eval()
    fin_outputs=[]
    with torch.no_grad():
      for _, data in enumerate(test_loader, 0):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        outputs = model(ids, mask, token_type_ids)
        fin_outputs.extend(outputs.cpu().detach().numpy().tolist())
    return fin_outputs

In [ ]:
outputs=test()
outputs = np.clip(np.sign(np.array(outputs)), a_min=0,a_max=None)
predictions_db = pd.DataFrame(outputs,  columns=['upr','ss','ps','av','m','c','pm'])
predictions_db = predictions_db.apply(lambda x:  list(map(int,x)))


In [ ]:
predictions_db

,upr,ss,ps,av,m,c,pm
0,0,1,0,0,0,0,0
1,1,0,0,1,1,0,0
2,1,1,0,0,0,0,0
3,0,1,0,0,0,0,0
4,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...
3827,0,1,0,0,1,0,0
3828,0,1,0,0,0,0,0
3829,0,1,0,1,1,0,0
3830,1,1,0,0,1,0,0


In [ ]:
z=np.load('zero.npz')
ze=z['arr_0']
zero=list(map(int,ze))

In [ ]:
for i in predictions_db.index:
  if i in zero:
    predictions_db.loc[i,'upr']=0
    predictions_db.loc[i,'ss']=0
    predictions_db.loc[i,'ps']=0
    predictions_db.loc[i,'av']=0
    predictions_db.loc[i,'m']=0
    predictions_db.loc[i,'c']=0
    predictions_db.loc[i,'pm']=0
    


In [ ]:
predictions_db.to_csv('./final_res/task2.txt', index=False, sep=',', header=False)

In [ ]:
from numpy import loadtxt
lines = loadtxt("task2_baseline_submission.txt", comments="#", unpack=False)

ValueError: ignored

In [ ]:
import csv
t=[]
with open("task2_baseline_submission.txt", 'r') as fd:
    reader = csv.reader(fd)
    for row in reader:
      row=[int(i) for i in row]
      t.append(row)
        

In [ ]:
t=np.array(t)

In [ ]:
t.shape

(2094, 7)

In [ ]:
actual=[]
for i in val_df.label:
  arr=[]
  for j in val_df.label[i]:
    arr.append(int(j))
  actual.append(arr)


TypeError: ignored

In [ ]:
val_df=pd.read_csv('task2_redefined_test.csv')
actual=val_df.label.values

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

In [ ]:
micro = metrics.f1_score(actual, outputs, average='micro')
macro = metrics.f1_score(autual, outputs, average='macro')
print(micro)
print(macro)

ValueError: ignored

In [ ]:
############# corr loss#########

In [ ]:
import pandas as pd
import numpy as np
import pickle
from ast import literal_eval

In [ ]:
train_df=pd.read_csv('task2_train_pmaug.csv')
target_cols=['upr','ss','ps','av','m','c','pm']

In [ ]:
def transform_labels(cat_list):
  label_ids = list(set(ind for ind,label in enumerate(cat_list) if label==1))
  return label_ids

In [ ]:
labels=[]
for label in train_df[target_cols].values:
  label=transform_labels(label)
  labels.append(label)


In [ ]:
NUM_CLASSES=7
thresh=0.1

In [ ]:
co_mat = np.zeros((NUM_CLASSES, NUM_CLASSES))
freqs = np.zeros(NUM_CLASSES)
for t_cats in labels:
	for i in range(len(t_cats)):
		freqs[t_cats[i]] += 1
		for j in range(i+1, len(t_cats)):
			if t_cats[i] < t_cats[j]:
				co_mat[t_cats[i], t_cats[j]] += 1
			else:
				co_mat[t_cats[j], t_cats[i]] += 1

for i in range(1, NUM_CLASSES):
	for j in range(0, i):
		co_mat[i,j] = co_mat[j,i]

#P(j|i) = co_mat[i,j]
for i in range(NUM_CLASSES):
	co_mat[i,:] = co_mat[i,:]/freqs[i]

uncorrelated_c_pairs = []

for i in range(NUM_CLASSES):
	for j in range(NUM_CLASSES):
		if (co_mat[i,j] <= thresh) and (i != j):
			uncorrelated_c_pairs.append((i,j,co_mat[i,j]))
	co_mat[i,i] = 1



In [ ]:
co_mat

array([[1.        , 0.67355372, 0.08264463, 0.37190083, 0.41322314,
        0.20661157, 0.26033058],
       [0.22925457, 1.        , 0.22362869, 0.17580872, 0.35302391,
        0.17018284, 0.21518987],
       [0.11049724, 0.87845304, 1.        , 0.06629834, 0.25414365,
        0.12154696, 0.09944751],
       [0.34749035, 0.48262548, 0.04633205, 1.        , 0.55984556,
        0.23552124, 0.34749035],
       [0.21551724, 0.54094828, 0.09913793, 0.3125    , 1.        ,
        0.31681034, 0.23275862],
       [0.22522523, 0.54504505, 0.0990991 , 0.27477477, 0.66216216,
        1.        , 0.32432432],
       [0.20588235, 0.5       , 0.05882353, 0.29411765, 0.35294118,
        0.23529412, 1.        ]])

In [ ]:
uncorrelated_c_pairs

[(0, 2, 0.08264462809917356),
 (2, 3, 0.06629834254143646),
 (2, 6, 0.09944751381215469),
 (3, 2, 0.04633204633204633),
 (4, 2, 0.09913793103448276),
 (5, 2, 0.0990990990990991),
 (6, 2, 0.058823529411764705)]

In [ ]:
filename = "%scorr_fuzzy_%s.pickle" % ('./saved_models/', thresh)
with open(filename, 'wb') as f:
    pickle.dump(uncorrelated_c_pairs, f)

filename = "%snorm_corr_mat.pickle" % ('./saved_models/')
with open(filename, 'wb') as f:
    pickle.dump(co_mat, f)